# Data Science Project Spring 2023

## 200+ Financial Indicators of US stocks (2014-2018)

### Yiwei Gong, Janice Herman, Alexander  Morawietz and Selina Waber

University of Zurich, Spring 2023

## Importing Packages

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pandas_datareader import data


from sklearn.model_selection import train_test_split

## Loading the Data Set


We used the data set from Nicolas Carbone from the webpage [kaggle](https://www.kaggle.com/datasets/cnic92/200-financial-indicators-of-us-stocks-20142018). Each dataset contains over 200 financial indicators, that are found in the [10-K filings](https://www.investopedia.com/terms/1/10-k.asp#:~:text=Key%20Takeaways-,A%2010%2DK%20is%20a%20comprehensive%20report%20filed%20annually%20by,detailed%20than%20the%20annual%20report.) of publicly traded companies from the US between the years 2014 - 2018.

In [ ]:
def load_dataset():
    project_directory = sys.path[0] ## get path of project directory
    data_directory = os.path.join(project_directory, 'data')

    years = [2014, 2015, 2016, 2017, 2018]

    ## Loading the yearly dataset into the array dfs
    dfs = []
    for year in years:
        df = pd.read_csv(os.path.join(data_directory, f'{year}_Financial_Data.csv'), sep=',')
        df['year'] = np.full(df.shape[0], str(year)) ## append column with the year respecitvely
        df['PRICE VAR [%]'] = df[f'{year +1} PRICE VAR [%]'] ## Adding variable of the same name for all df, e.g. '2016 PRICE VAR [%]' renamed to 'PRICE VAR [%]'
        df = df.drop(columns=[f'{year +1} PRICE VAR [%]']) # dropp year-specific variable name
        df.columns.values[0] = 'Stock Name' # name the first variable 
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True) ## concat the diffrent dataframes

    return df

## Some Explanation of Variables:

### Adding `year` as a categorical variable

We added a column named year which contains the respecitve year.

### Handling the variable `Price VAR [%]`

The last column, `PRICE VAR [%]`, lists the percent price variation of each stock for the year. For example, if we consider the dataset 2015_Financial_Data.csv, we will have:

- 200+ financial indicators for the year 2015;
- percent price variation for the year 2016 (meaning from the first trading day on Jan 2016 to the last trading day on Dec 2016).

We renamed all the variables with the specific year in it, e.g. `2016 PRICE VAR [%]` to `PRICE VAR [%]`. We dropped the old ones. 

### the variable `class`

class lists a binary classification for each stock, where

- for each stock, if the PRICE VAR [%] value is positive, class = 1. From a trading perspective, the 1 identifies those stocks that an hypothetical trader should BUY at the start of the year and sell at the end of the year for a profit.
- for each stock, if the PRICE VAR [%] value is negative, class = 0. From a trading perspective, the 0 identifies those stocks that an hypothetical trader should NOT BUY, since their value will decrease, meaning a loss of capital.

The columns `PRICE VAR [%]` and `class` make possible to use the datasets for both classification and regression tasks:

- If the user wishes to train a machine learning model so that it learns to classify those stocks that in buy-worthy and not buy-worthy, it is possible to get the targets from the class column;

- If the user wishes to train a machine learning model so that it learns to predict the future value of a stock, it is possible to get the targets from the PRICE VAR [%] column.

### the variable `Stock Name`

We named the first variable Stock Namesince it has not been named in the original dataset.


## Numerical and Catgorical Features/Variables



In [ ]:
# We are converting Classto a cathegorical variable.
def class_to_categorical(df):
    df['class'] = df.Class.astype('object') ## object or catheogry?? whats the difference??
    return df

In [ ]:
def print_number_of_numerical_categorical_variables(df):
    
    numCols = df.select_dtypes(exclude='object').columns
    print(f"There are {len(numCols)} numerical features:\n")

    catCols = df.select_dtypes(include='object').columns
    print(f"There are {len(catCols)} categorical features:\n", catCols)

## Any Duplicates? 

No, there are no duplicates for rows but there are 20 duplicates for columns/ 10 each. Not same variable name but same data!

In [ ]:
def check_duplicates_row(df):
    print(f'Duplicates in Rows:', True in list(df.duplicated()))

In [ ]:
def check_duplicates_col(df):
    print(f'Duplicates in Columns:', True in list(df.T.duplicated().T))
    print("Show the Duplicates:")
    print(df.T[df.T.duplicated(keep=False)].T)

In [ ]:
def remove_duplicates(df,columns):
    shape_old=df.shape

    df=df.drop(columns=columns)

    print(f' Shape with duplicates:', shape_old) 
    print(f' Shape after removal of duplicates:', df.shape) 
    
    return df


Our Duplicates are the following pairs:

- `ebitperRevenue` and `eBITperRevenu`
- `ebtperEBIT` and `eBTperEBIT`
- `niperEBT` and `nIperEBT`
- `returnOnAssets` and `Return on Tangible Assets`
- `returnOnCapitalEmployed` and `ROIC`
- `payablesTurnover` and `Payables Turnover`
- `inventoryTurnover` and `Inventory Turnover`
- `debtRatio` and `Debt to Assets`
- `debtEquityRatio` and `Debt to Equity`
- `cashFlowToDebtRatio` and `cashFlowCoverageRatios`

## Feature Engineering

We got the data from the following webpages: [S&P means](https://www.macrotrends.net/2526/sp-500-historical-annual-returns) and [inflation](https://www.macrotrends.net/countries/USA/united-states/inflation-rate-cpi)

In [ ]:
def adding_indicators(df):
    
    ## Yearly Means of S&P 500
    sp500_means = pd.Series([11.39, -0.73, 9.54, 19.42, -6.24], index = [2014, 2015, 2016, 2017, 2018]) ## or should it start with year 2015 to year 2019???
    ## for year 2019 we got 28.88%
    
    # Yearly Inflaction Rate measured by consumer price index
    inflation = pd.Series([1.62, 0.12, 1.26, 2.13, 2.44], index = [2014, 2015, 2016, 2017, 2018]) ## or should it start with year 2015 to year 2019???
    ## or should we look at annual change????
    
    ##Adding to the dataframe
    df["inflation"] = df.apply(lambda x: inflation[int(x["year"])], axis=1)
    df["sp500_means"] = df.apply(lambda x: sp500_means[int(x["year"])], axis=1)
    
    
    ## Calculation of Excess Return:
    df["excess_return"] = np.subtract(df["PRICE VAR [%]"], df["sp500_means"])


    ## Calculation of Cashflow Margin:
    df["cashflow_margin"] = df["Operating Cash Flow"].divide(df["Revenue"])
    # Pay attention to ZeroDivisionError, replace infinity by NAN
    df["cashflow_margin"] = df["cashflow_margin"].replace([np.inf, -np.inf], np.nan)
   
    
    ## Calculation of Return on Net Assets (RONA)
    df["Net_working_capital"] = df["Total assets"]-df["Cash and cash equivalents"]
    df["RONA"] = df["EBIT"]/df["Net_working_capital"]
    df["RONA"] = df["RONA"].replace([np.inf, -np.inf], np.nan)


    #df["operating_liabilities"] = df["Total liabilities"]-df["Total debt"]
    
    return df

In [ ]:
def dropping_indicators(df):
    df_new = df.drop(["operatingProfitMargin"], axis = 1) # consisting only of the value 1.
    
    #maybe more to drop? which are not yet addressed in correlation or duplicates or elsewhere?
    
    return df_new

## Correlation of the variables

In [ ]:
def show_correlation(df):
    X = df[df.columns.difference(['Class', 'Stock Name', 'Sector', 'year', 'PRICE VAR [%]'])]
    y = df["Class"]
    plt.matshow(X.corr().abs())
    plt.colorbar()
    plt.show()

    abs_corr = X.corr().abs()
    for i in range(len(abs_corr)):
        abs_corr.iloc[i, i] = 0
        
    abs_corr_unstack = abs_corr.unstack()
    abs_corr_unstack.sort_values(kind="quicksort")[-50:]

    print((abs_corr_unstack.values>0.99).sum()/2)

    return abs_corr_unstack

In [ ]:
#suggestion to deal with the correlations: remove a variable if its correlation with another variable is higher than 0.99
def remove_correlation(df, abs_corr_unstack):
    columns_to_drop = []
    columns_to_remain = []

    for pair in abs_corr_unstack.index.values:
        if abs_corr_unstack[pair] > 0.99:
            if pair[0] not in columns_to_remain and pair[1] not in columns_to_remain:
                    columns_to_remain.append(pair[0])
                    if pair[1] not in columns_to_drop:
                        columns_to_drop.append(pair[1])
            elif pair[0] in columns_to_remain:
                if pair[1] not in columns_to_drop:
                    columns_to_drop.append(pair[1])
            elif pair[1] in columns_to_remain:
                if pair[0] not in columns_to_drop:
                    columns_to_drop.append(pair[0])

    df_corr_removed = df.drop(columns=columns_to_drop)

    return df_corr_removed

## Class Balance?

The Variable `Class`is not balanced. We have to keep that in mind for train and test split. 

In [ ]:
def check_class_imbalance(y):
    sns.countplot(x=y)

## Outliers Dedection for `PRICE VAR[%]`

In [ ]:
def get_list_of_sectors(df):
    df_ = df.loc[:, ['Sector','PRICE VAR [%]']]

    # Get list of sectors
    sector_list = df_['Sector'].unique()

    # Plot the percent price variation for each sector
    for sector in sector_list:
        
        temp = df_[df_['Sector'] == sector]

        plt.figure(figsize=(30,5))
        plt.plot(temp['PRICE VAR [%]'])
        plt.title(sector.upper())
        plt.show()
    

### Outliers

 copy paste from here https://www.kaggle.com/code/cnic92/explore-and-clean-financial-indicators-dataset

In [ ]:
def check_outliers(df):
    # Get stocks that increased more than 500%
    gain = 500
    top_gainers = df[df['PRICE VAR [%]'] >= gain]
    top_gainers = top_gainers['PRICE VAR [%]'].sort_values(ascending=False)
    print(f'{len(top_gainers)} STOCKS with more than {gain}% gain.')



## Outliers cleaning

There are outliers/extreme values that are probably caused by mistypings. During our analysis of the data, we noticed that the values of NA and 0 were frequently used. We realized that 0 was used interchangeably with NA.  Also there are a lot of values that seem impossible. 

In [ ]:
"""
TODO!
"""

def remove_outliers(df):
    # IQR
    test= df[['ebtperEBIT', 'returnOnCapitalEmployed']]
    for col in test:
        iqr= stats.iqr(df[col], nan_policy='omit')
        print(iqr)

    ## Z-Score: 
    threshold = 3

    test= df[['ebtperEBIT', 'returnOnCapitalEmployed']]
    for col in test:
        z_score= stats.zscore(df[col], nan_policy='omit')
        outlier_indices = np.where(z_score > threshold)
        print(outlier_indices)

    # IQR
    test= df[['ebtperEBIT', 'returnOnCapitalEmployed']]
    for col in test:
        iqr= stats.iqr(df[col], nan_policy='omit')
        print(iqr)

    #''''''
    

    return df


https://www.kaggle.com/code/nareshbhat/outlier-the-silent-killer

## Missing Values

There are a lot of missing values. 

In [ ]:
def check_missing_values(df):
    print(f'There are in total {df.isnull().sum().sum()} NAN in the dataframe')

    ## Overview of all variables with missing values
    df.isnull().mean().sort_values(ascending=False).plot.bar(figsize=(100,20))
    plt.ylabel('Percentage of missing values')
    plt.xlabel('Variables')
    plt.title('Quantifying ALL missing data')
    plt.show()

    most_nan = df.isnull().mean().sort_values(ascending=False)
    most_nan = most_nan[most_nan > 0.3]

    most_nan.plot.bar(figsize=(20,20))
    plt.ylabel('Percentage of missing values')
    plt.xlabel('Variables')
    plt.title('Data with more than 30% missing')
    plt.show()

    # Percentage of missing values for the variables
    missing = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([missing, percent], axis=1, keys=['Nr. of missing values', 'Percent of Missing Values'])
    missing_data.head(25)

    # Plot missing values 2.0
    sns.heatmap(df.isna().transpose(), cmap="Blues", cbar_kws={'label': 'Missing Values'});


## Handling Missing Data

In [ ]:
def handle_missing_data(df, threshold):

    print(sum(df.isna().mean() > threshold)) # 76 of the remaining variables have more than 30% NAs
    df = df.loc[::, df.isna().mean() < threshold] # drop all columns with NA proportion higher than threshold

    numCols = df.select_dtypes(include=['float64', 'int64']).columns
    print("New numerical columns:", numCols)
    df[numCols] = df[numCols].fillna(df[numCols].median())

    catCols = df.select_dtypes(exclude=np.number).columns
    print("New categorical columns:", catCols)
    for col in catCols:
        df[col].fillna("Unknown", inplace=True)

    return numCols, catCols, df

## Handling unique values and cardinality


"Each categorical variable consists of unique values. A categorical feature is said to possess high cardinality when there are too many of these unique values. One-Hot Encoding becomes a big problem in such a case since we have a separate column for each unique value (indicating its presence or absence) in the categorical variable. This leads to two problems, one is obviously space consumption, but this is not as big a problem as the second problem, the curse of dimensionality" [reference here](https://towardsdatascience.com/dealing-with-features-that-have-high-cardinality-1c9212d7ff1b)


In [ ]:
def reduce_cardinality(column, threshold):
    #threshold
    threshold_value = int(threshold * len(column))
    # Initialize
    categories_list = []
    s = 0
    counts = []
    
    # Count the frequencies of unique values in the column
    for value in column:
        # Check if the value is already in the counts list
        index = next((i for i, x in enumerate(counts) if x[0] == value), None)
        if index is not None:
            counts[index] = (value, counts[index][1] + 1)
        else:
            counts.append((value, 1))
    
    # Sort the list of tuples based on count in descending order
    counts.sort(key=lambda x: x[1], reverse=True)
    
    # Loop through the tuples (value, count)
    for i, j in counts:
        # Add the frequency to the global sum
        s += j
        # Append the category name to the list
        categories_list.append(i)
        # Check if the global sum has reached the threshold value, if so break the loop
        if s >= threshold_value:
            break
    
    # Append the category 'Other' to the list
    categories_list.append('Other')
    
    # Replace all instances not in our new categories by 'Other'
    new_column=column.apply(lambda x: x if x in categories_list else 'Other')
    
    return new_column, categories_list


In [ ]:
#Testing

#loading dataset
df=load_dataset()

#Call the function with a default threshold of 75%
transformed_column,new_category_list=reduce_cardinality(df['Sector'], threshold=0.75)
print(transformed_column)
print(new_category_list)


## Adding Dummies

In [ ]:
def add_dummies(df, catCols):
    df = pd.get_dummies(df, columns=catCols)
    df.head()

    return df

## Get Most Significant Features

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

def get_significant_features(X_train, X_test, y_train, n):
    # Feature selection using Extra Trees Classifier on the resampled training data
    model = ExtraTreesClassifier(random_state=42)
    model.fit(X_train, y_train)
    importances = model.feature_importances_

    # Select top features with highest importance scores
    top_features = pd.Series(importances, index=X_train.columns).nlargest(n)

    # Subset X_resampled and X_test with selected features
    X_train_selected = X_train[top_features.index]
    X_test_selected = X_test[top_features.index]

    return X_train_selected, X_test_selected

## Test SVM

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC

In [ ]:
# load dataset
df = load_dataset()
df.head()

In [ ]:
# check_duplicates_row(df)
# check_duplicates_col(df)

In [ ]:
# Remove duplicated columns
duplicated_columns = ['eBITperRevenue', 'eBTperEBIT', 'nIperEBT', 'Return on Tangible Assets', 
                     'ROIC', 'Payables Turnover', 'Inventory Turnover', 'Debt to Assets', 'Debt to Equity', 
                     'cashFlowCoverageRatios']
df = remove_duplicates(df, duplicated_columns)
df.head()

In [ ]:
# check correlation
abs_corr_unstack = show_correlation(df)

In [ ]:
# Remove correlation
df = remove_correlation(df, abs_corr_unstack)
df.head()

In [ ]:
# Check missing values
# check_missing_values(df)

In [ ]:
# Remove missing values
numCols, catCols, df = handle_missing_data(df, 0.3)
df.head()

In [ ]:
# add dummies
df_dummies = add_dummies(df, catCols)
df_dummies.head

In [ ]:
X = df_dummies[df.columns.difference(['Class', 'Stock Name', 'Sector', 'year', 'PRICE VAR [%]'])]
y = df['Class']

## Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state = 42) 

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Select most significant features
X_train_selected, X_test_selected = get_significant_features(X_train, X_test, y_train, 300)

In [ ]:
# Create StandardScaler object
sc = StandardScaler()

# Standardize features; equal results as if done in two
X_train_std = sc.fit_transform(X_train_selected)
X_test_std = sc.transform(X_test_selected)

In [ ]:
# Grid Search for getting optimal C and gamma
gamma_range = np.outer(np.logspace(-3, 0, 4),np.array([1,5]))
gamma_range = gamma_range.flatten()
print(gamma_range)

C_range = np.outer(np.logspace(-1, 1, 3),np.array([1,5]))
C_range = C_range.flatten()
print(C_range)

parameters = {'kernel':['linear', 'rbf'], 'C':C_range, 'gamma': gamma_range}

svm = SVC()
grid = RandomizedSearchCV(estimator=svm, param_distributions=parameters, n_iter=5, n_jobs=-1, verbose=2)
grid.fit(X_train_std, y_train)

print('Best CV accuracy: {:.2f}'.format(grid.best_score_))
print('Test score:       {:.2f}'.format(grid.score(X_test_std, y_test)))
print('Best parameters: {}'.format(grid.best_params_))